# Regression!

Lets try and predict the next value in a linear series of values using regression! We are going to use the MPG dataset

## Imports

In [3]:
import pandas as pd
import seaborn as sns
import tensorflow as tf

ModuleNotFoundError: No module named 'unittest'

## Looking at our data

In [ ]:
df = pd.read_csv("auto-mpg.csv")
df